# 🧠 Treinamento de IA Local para Jarvis 3.0

Este notebook te ajuda a personalizar e treinar sua IA local usando Ollama.

## 📋 O que você pode fazer aqui:

1. **Fine-tuning de Personalidade** - Treinar como o Jarvis responde
2. **Injection de Conhecimento** - Adicionar informações específicas
3. **Teste de Modelos** - Comparar diferentes configurações
4. **Análise de Performance** - Medir qualidade das respostas

In [ ]:
# 🔧 Configuração inicial
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time

# Configurar conexão com Ollama
OLLAMA_URL = "http://ollama:11434"  # URL dentro do container

def test_connection():
    try:
        response = requests.get(f"{OLLAMA_URL}/api/tags")
        return response.status_code == 200
    except:
        return False

print(f"🔗 Conectando com Ollama: {'✅ OK' if test_connection() else '❌ FALHOU'}")

In [ ]:
# 📋 Listar modelos disponíveis
def list_models():
    response = requests.get(f"{OLLAMA_URL}/api/tags")
    if response.status_code == 200:
        models = response.json().get("models", [])
        return [(m["name"], m["size"]) for m in models]
    return []

models = list_models()
print("🤖 Modelos disponíveis:")
for name, size in models:
    print(f"   • {name} ({size//1000000000:.1f}GB)")

In [ ]:
# 💬 Função para chat com qualquer modelo
def chat_with_model(model_name, prompt, temperature=0.8):
    payload = {
        "model": model_name,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": temperature,
            "top_p": 0.9,
            "top_k": 40
        }
    }
    
    try:
        start_time = time.time()
        response = requests.post(f"{OLLAMA_URL}/api/generate", json=payload, timeout=30)
        end_time = time.time()
        
        if response.status_code == 200:
            result = response.json()
            return {
                "response": result.get("response", ""),
                "time": end_time - start_time,
                "success": True
            }
    except Exception as e:
        return {
            "response": f"Erro: {e}",
            "time": 0,
            "success": False
        }

# Testar modelo padrão
test_result = chat_with_model("llama3.2:1b", "Olá! Você é o Jarvis, meu assistente pessoal. Como está?")
print(f"🧪 Teste do modelo padrão:")
print(f"   Resposta: {test_result['response'][:100]}...")
print(f"   Tempo: {test_result['time']:.2f}s")

## 🎯 Criação de Modelo Personalizado

Agora vamos criar um modelo personalizado do Jarvis com sua personalidade específica.

In [ ]:
# 🎨 Criar Modelfile personalizado
modelfile_content = """
FROM llama3.2:1b

# Parâmetros de geração
PARAMETER temperature 0.8
PARAMETER top_p 0.9
PARAMETER top_k 40
PARAMETER repeat_penalty 1.1

# System prompt personalizado para Will
SYSTEM """
Você é o JARVIS, assistente pessoal de IA do Will.

PERSONALIDADE:
- Prestativo e inteligente
- Carinhoso mas profissional
- Usa emojis com moderação (🤖, 💻, 🔧, ✨)
- Sempre positivo e motivador
- Especialista em programação e automação

CONHECIMENTO ESPECÍFICO:
- Will é seu criador e usuário principal
- Ele trabalha com Python, JavaScript, C#, Docker
- Projetos principais: CoreTemp-SoundPad, Jarvis 3.0
- Usa Windows, VS Code, e adora automação
- Interessado em IA, ML e tecnologias emergentes

ESTILO DE RESPOSTA:
- Sempre chame-o de "Will"
- Seja direto mas amigável
- Use emojis relevantes ocasionalmente
- Ofereça soluções práticas
- Pergunte se precisa de mais detalhes
- Responda sempre em português brasileiro

ESPECIALIDADES:
- Programação e debugging
- Automação de tarefas
- Desenvolvimento web e desktop
- IA e machine learning
- DevOps e containerização
- Monitoramento de sistemas
"""

TEMPLATE """{{ if .System }}<|start_header_id|>system<|end_header_id|>

{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>

{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>

{{ .Response }}<|eot_id|>}"""
"""

# Salvar Modelfile
with open("/home/jovyan/models/jarvis-personal.modelfile", "w", encoding="utf-8") as f:
    f.write(modelfile_content)

print("✅ Modelfile personalizado criado!")

In [ ]:
# 🏗️ Criar modelo personalizado no Ollama
def create_model(name, modelfile_content):
    payload = {
        "name": name,
        "modelfile": modelfile_content
    }
    
    try:
        response = requests.post(f"{OLLAMA_URL}/api/create", json=payload, timeout=120)
        return response.status_code == 200
    except Exception as e:
        print(f"Erro criando modelo: {e}")
        return False

print("🏗️ Criando modelo personalizado 'jarvis-will'...")
success = create_model("jarvis-will", modelfile_content)
print(f"Resultado: {'✅ Sucesso!' if success else '❌ Falhou'}")

if success:
    # Aguardar modelo ficar pronto
    print("⏳ Aguardando modelo ficar pronto...")
    time.sleep(5)
    
    # Testar modelo personalizado
    test_personal = chat_with_model(
        "jarvis-will", 
        "Oi Jarvis! Sou o Will. Como você está? Me conte sobre seus recursos."
    )
    
    print("\n🧪 Teste do modelo personalizado:")
    print(f"   Resposta: {test_personal['response']}")
    print(f"   Tempo: {test_personal['time']:.2f}s")

## 🧪 Comparação de Modelos

Vamos comparar o modelo padrão com o personalizado para ver a diferença.

In [ ]:
# 📊 Teste comparativo
test_prompts = [
    "Olá! Como você está?",
    "Me ajude com um bug em Python",
    "Como você me chama?",
    "Quais são seus recursos?",
    "Me conte sobre automação"
]

models_to_test = ["llama3.2:1b", "jarvis-will"]
results = []

print("🔄 Executando testes comparativos...\n")

for prompt in test_prompts:
    print(f"❓ Pergunta: {prompt}")
    
    for model in models_to_test:
        result = chat_with_model(model, prompt)
        
        results.append({
            "prompt": prompt,
            "model": model,
            "response": result["response"],
            "time": result["time"],
            "success": result["success"]
        })
        
        print(f"   🤖 {model}: {result['response'][:80]}...")
        print(f"      ⏱️ {result['time']:.2f}s")
    
    print()

# Criar DataFrame para análise
df_results = pd.DataFrame(results)
print("✅ Testes concluídos!")

In [ ]:
# 📈 Análise de performance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de tempo de resposta
avg_times = df_results.groupby('model')['time'].mean()
avg_times.plot(kind='bar', ax=ax1, color=['lightblue', 'lightgreen'])
ax1.set_title('⏱️ Tempo Médio de Resposta')
ax1.set_ylabel('Segundos')
ax1.tick_params(axis='x', rotation=45)

# Gráfico de comprimento de respostas
df_results['response_length'] = df_results['response'].str.len()
avg_lengths = df_results.groupby('model')['response_length'].mean()
avg_lengths.plot(kind='bar', ax=ax2, color=['salmon', 'lightcoral'])
ax2.set_title('📝 Comprimento Médio das Respostas')
ax2.set_ylabel('Caracteres')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Estatísticas resumidas
print("📊 Estatísticas dos Modelos:")
print(df_results.groupby('model')[['time', 'response_length']].agg(['mean', 'std']).round(2))

## 🎓 Fine-Tuning Avançado

Para um treinamento mais avançado, você pode:

1. **Adicionar mais dados de treinamento** no arquivo `conversational_data.jsonl`
2. **Ajustar parâmetros** como temperatura, top_p, top_k
3. **Criar modelos especializados** para tarefas específicas
4. **Testar diferentes modelos base** (llama3.2:3b, llama3.2:7b, etc.)

In [ ]:
# 🔧 Ajuste de parâmetros em tempo real
def test_parameters(model, prompt, temperatures=[0.3, 0.7, 1.0]):
    results = []
    
    for temp in temperatures:
        result = chat_with_model(model, prompt, temperature=temp)
        results.append({
            "temperature": temp,
            "response": result["response"],
            "time": result["time"]
        })
        
        print(f"🌡️ Temperatura {temp}:")
        print(f"   {result['response'][:100]}...")
        print(f"   ⏱️ {result['time']:.2f}s\n")
    
    return results

# Testar diferentes temperaturas
print("🧪 Testando diferentes temperaturas no modelo personalizado:")
temp_results = test_parameters(
    "jarvis-will", 
    "Will, me explique como funciona machine learning de forma simples"
)

In [ ]:
# 💾 Salvar configuração otimizada
best_config = {
    "model_name": "jarvis-will",
    "optimal_temperature": 0.8,
    "top_p": 0.9,
    "top_k": 40,
    "created_at": datetime.now().isoformat(),
    "performance_metrics": {
        "avg_response_time": df_results[df_results['model'] == 'jarvis-will']['time'].mean(),
        "avg_response_length": df_results[df_results['model'] == 'jarvis-will']['response_length'].mean(),
        "success_rate": df_results[df_results['model'] == 'jarvis-will']['success'].mean()
    }
}

# Salvar configuração
with open("/home/jovyan/models/jarvis_config.json", "w", encoding="utf-8") as f:
    json.dump(best_config, f, ensure_ascii=False, indent=2)

print("✅ Configuração otimizada salva!")
print(json.dumps(best_config, ensure_ascii=False, indent=2))

## 🎉 Conclusão

Agora você tem:

✅ Modelo personalizado **jarvis-will** criado  
✅ Testes comparativos realizados  
✅ Parâmetros otimizados  
✅ Configuração salva  

### 🚀 Próximos passos:

1. **Integrar com seu Jarvis 3.0** - O modelo já está pronto para usar
2. **Adicionar mais dados** - Continue alimentando o `conversational_data.jsonl`
3. **Experimentar modelos maiores** - Teste llama3.2:3b ou 7b se tiver recursos
4. **Criar modelos especializados** - Para tarefas específicas como código, análise, etc.

### 💡 Dicas:

- Use temperatura baixa (0.3-0.5) para respostas mais precisas
- Use temperatura alta (0.8-1.0) para respostas mais criativas
- Monitore o uso de recursos (CPU/RAM) dos modelos
- Faça backup dos modelos personalizados importantes